In [1]:
# Import packages
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

import nltk

## Data import

In [5]:
df = pd.read_csv('Magic_Castle_Reviews.csv', index_col="Unnamed: 0").reset_index(drop=True)
df.head()

,Date,Review,Rating
0,9/11/2019,"So this place gets a lot of love, and a bit of...",4 star rating
1,11/1/2020,this was my favorite place to visit by far oth...,5 star rating
2,2/9/2020,"The Magic Castle (Hollywood, CA)I had this pla...",4 star rating
3,1/23/2020,I recently had the pleasure of visiting the Ma...,4 star rating
4,7/22/2019,So i learned a couple more new things througho...,5 star rating


In [6]:
# Convert to list
data = df.Review.values.tolist()

In [7]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

## Tokenization

In [8]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['so', 'this', 'place', 'gets', 'lot', 'of', 'love', 'and', 'bit', 'of', 'hate', 'too', 'didn', 'love', 'it', 'but', 'definitely', 'didn', 'hate', 'it', 'it', 'an', 'interesting', 'place', 'for', 'sure', 'the', 'building', 'itself', 'is', 'gorgeous', 'it', 'beautiful', 'on', 'the', 'outside', 'and', 'spectacular', 'on', 'the', 'inside', 'pretty', 'sure', 'read', 'that', 'the', 'building', 'itself', 'is', 'over', 'years', 'old', 'the', 'business', 'inside', 'the', 'magic', 'castle', 'has', 'been', 'operating', 'for', 'decades', 'know', 'they', 'call', 'themselves', 'private', 'club', 'and', 'you', 'have', 'to', 'be', 'invited', 'to', 'get', 'in', 'or', 'stay', 'at', 'the', 'magic', 'castle', 'hotel', 'next', 'door', 'like', 'we', 'did', 'but', 'personally', 'find', 'the', 'admissions', 'policy', 'bit', 'obnoxious', 'it', 'certainly', 'didn', 'feel', 'like', 'private', 'club', 'but', 'it', 'is', 'what', 'it', 'is', 'everyone', 'has', 'tips', 'on', 'how', 'to', 'get', 'an', 'invitation',

In [10]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [14]:
bigram[data_words[0]]

['so',
 'this',
 'place',
 'gets',
 'lot',
 'of',
 'love',
 'and',
 'bit',
 'of',
 'hate',
 'too',
 'didn',
 'love',
 'it',
 'but',
 'definitely',
 'didn',
 'hate',
 'it',
 'it',
 'an',
 'interesting',
 'place',
 'for',
 'sure',
 'the',
 'building',
 'itself',
 'is',
 'gorgeous',
 'it',
 'beautiful',
 'on',
 'the',
 'outside',
 'and',
 'spectacular',
 'on',
 'the',
 'inside',
 'pretty',
 'sure',
 'read',
 'that',
 'the',
 'building',
 'itself',
 'is',
 'over',
 'years',
 'old',
 'the',
 'business',
 'inside',
 'the',
 'magic',
 'castle',
 'has',
 'been',
 'operating',
 'for',
 'decades',
 'know',
 'they',
 'call',
 'themselves',
 'private_club',
 'and',
 'you',
 'have',
 'to',
 'be',
 'invited',
 'to',
 'get',
 'in',
 'or',
 'stay',
 'at',
 'the',
 'magic',
 'castle',
 'hotel',
 'next',
 'door',
 'like',
 'we',
 'did',
 'but',
 'personally',
 'find',
 'the',
 'admissions',
 'policy',
 'bit',
 'obnoxious',
 'it',
 'certainly',
 'didn',
 'feel',
 'like',
 'private_club',
 'but',
 'it',
 

In [ ]:
# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

## Remove Stopwords, Make Bigrams and Lemmatize

In [ ]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

## Create the Dictionary and Corpus needed for Topic Modeling

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

In [ ]:
# What is the first word?
id2word[0]

In [ ]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

## Building the Topic Model

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

## Model fit

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

## Viz time!

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis